Hack days, Lund, 2018, May 30-31
=================
*Where to put one’s data* - story by Michael C. Witt
---------------------

**example prototyping notebooks
free to experiment, free to share**

In [178]:
# using 'request' for communication with re3data via its RestAPI
import requests
# lxml for parsing RestAPI xml replies
import lxml.etree as etree
from lxml import html

Example 1
---------

get list of all repositories

In [13]:
r = requests.get('https://www.re3data.org/api/v1/repositories')
print(r.text)

<?xml version="1.0" encoding="UTF-8"?>
<list>
            <repository>
            <id>r3d100000002</id>
            <name>Access to Archival Databases</name>
            <link href="/api/v1/repository/r3d100000002" rel="self" />
        </repository>       
            <repository>
            <id>r3d100000004</id>
            <name>Datenbank Gesprochenes Deutsch</name>
            <link href="/api/v1/repository/r3d100000004" rel="self" />
        </repository>       
            <repository>
            <id>r3d100000005</id>
            <name>UNC Dataverse</name>
            <link href="/api/v1/repository/r3d100000005" rel="self" />
        </repository>       
            <repository>
            <id>r3d100000006</id>
            <name>Archaeology Data Service</name>
            <link href="/api/v1/repository/r3d100000006" rel="self" />
        </repository>       
            <repository>
            <id>r3d100000007</id>
            <name>ESO Science Archive Facility</name>
      

repo_keywords(rid)
----------------
Simple fuction to retrieve keywords for a re3data repository record

In [172]:
def repo_keywords(rid):
    keywords = ()
    r = requests.get('https://www.re3data.org/api/v1/repository/'+rid)
    tree = etree.fromstring(r.content)
    for elem_key in  tree.findall('{*}repository/{*}keyword'):
        keywords = keywords + (elem_key.text.lower(),)
    return keywords

# Test
repo_id = 'r3d100012674'
keywords = repo_keywords(repo_id)
print(keywords)

('climatology', 'marine data', 'chemistry', 'interpolated maps', 'european seas', 'bathymetry', 'salinity', 'climatology', 'plankton')


Example 2
--------

Let's play a little more. How many records are there in re3data ?

In [141]:
r = requests.get('https://www.re3data.org/api/beta/repositories')
tree = etree.fromstring(r.content)
target = tree.findall('{*}repository')
print("Total number of repos: %d" % (len(target),))

Total number of repos: 2090


And how many records are there in **'316 Geochemistry, Mineralogy and Crystallography'**?

In [160]:
domain = '316 Geochemistry, Mineralogy and Crystallography'

r = requests.get('https://www.re3data.org/api/beta/repositories?subjects[]='+domain)
tree = etree.fromstring(r.content)
target = tree.findall('{*}repository')
print("%s: %d repos" % (domain,len(target),))

316 Geochemistry, Mineralogy and Crystallography: 75 repos


So exactly as reported on the re3data web, see the page aslo for other domains:
TODO

repo_ids(domain)
----------------
Function to get all repositary ids for the given domain

In [ ]:
# get repos ids
def repo_ids(domain):
    rids = ()
    r = requests.get('https://www.re3data.org/api/beta/repositories?subjects[]='+domain)
    tree = etree.fromstring(r.content)
    target = tree.findall('{*}repository')
    for elem in target:
        rids = rids + (elem.findall('id')[0].text,)
    return rids

# get repository ids for the domain above
rids = repo_ids(domain)

We know all repository ids in the given domain, let's get the keywords associated with them in re3data

In [180]:
# find all repo-keywords in this domain
all_keywords = ()

for rid in rids:
    all_keywords = all_keywords + repo_keywords(rid)
    
print("Total no of keywords in domian: %" % (len(all_keywords),))

ConnectionError: HTTPSConnectionPool(host='www.re3data.org', port=443): Max retries exceeded with url: /api/v1/repository/r3d100010989 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x113ff0d10>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

Pick only unique ones (the simples method)

In [176]:
uniq_keywords = ()
for word in all_keywords:
    if word not in uniq_keywords:
        uniq_keywords = uniq_keywords + (word,)

print(uniq_keywords)

('petroleum', 'geological mapping', 'geophysical surveys', 'geochemical modeling', 'basins (geology)', 'oil wells', 'natural gas', 'hydrocarbons', 'seismic arrays', 'geology', 'zoology', 'paleontology', 'mineralogy', 'anthropology', 'paleoanatomy', 'botany', 'mycology', 'marine biology', 'marine geology', 'coastal geology', 'climate change', 'marine policy', 'seafloor mapping', 'sediment transport', 'tectonics', 'ecosystems', 'planetary scientists', 'x-ray spectroscopy', 'gemology', 'gemologists', 'mineralogists', 'x-ray diffraction', 'crystallography', 'raman spectrosocpy', 'electron microprobe', 'geochronology', 'zero-mg hydrothermal end-member compositions', 'dissolved gas', 'radiogenic istopes', 'stable istopes', 'key air pollutants', 'facility data', 'heavy metals', 'persistent organic pollutants', 'waste rock', 'tailings', 'acidic drainage', 'disposal', 'substance lists', 'mapping', 'samples', 'materials', 'methods', 'standards', 'strain and seismic borehole data', 'data librarie

We have some keywords ! :-)
    
TODO:
====

Now we need a "clustering" expert.

Write a function that will create 'word clouds' from this.
Change the domain e.g. to some loarge one: '3 - ' and try how it works.